## Guiding the rational design of biocompatible metal-organic frameworks for drug delivery using machine learning.
Dhruv Menon and David Fairen-Jimenez*<br/>
Email(s): dm958@cam.ac.uk, df334@cam.ac.uk<br/>

This Jupyter Notebook presents an accessible framework for predicting the biocompatibility of a MOF based on the toxicity profile of its linker. Discussions pertaining to the treatment of the metallic center are redirected to the associated manuscript.

The contents of this Jupyter Notebook are intentionally concise and straightforward, tailored towards readers less versed in code and machine learning. The code is in principle designed to process a CIF file of the relevant MOF, predicting the toxicity category of its constituent linker. However, since the fragmentation of the MOF is dependent on openbabel, which cannot be setup in this format, we operate under the assumption that the user knows the metal centre and the linker present in the MOF. Training the model, which exceeds 200 Mb, is executed within this notebook and typically concludes within a few minutes. It is noteworthy that aside from specifying the filepaths to the CIF file, no further modifications to the code are required. While the parameters employed reflect those identified as optimal during model hyperparameter optimization, readers are encouraged to make adjustments as deemed appropriate. In addition, we provide detailed comments to guide users with less experience in coding - to augment the interpretability of the notebook.

For individuals with a more advanced coding background or those seeking to execute the models in a high-throughput manner, the option to utilize the source code directly from the GitHub repository is available.

Should you incorporate these machine learning models into your work, we kindly request citation of the corresponding paper for proper acknowledgment.

### Loading all the dependencies
To execute the models, a set of dependencies is essential. In the following code block(s), we shall initiate the installation of these dependencies and subsequently load them. Should these dependencies already be installed, you may bypass the code block for their installation, marked by the command !pip.

- The **pandas** library is used for processing the training data for the machine learning models.
- The **numpy** library is used for some of the scientfic computational involved in data handling and model training.
- The **sklearn** library is used for developing the machine learning models.
- The **rdkit** library is used for extracting relevant features from organic molecules that will be passed to the machine learning models during the training procedure.
- The **matplotlib** & **seaborn** libraries are used for plotting data

Pay special attention to the version of pandas, numpy and scikit-learn. This is because some versions are incompatible with each other. If that is the case, you are most likely to end up getting errors. To prevent this, we install specific versions of these libraries which are compatible with each other. The models are tested and are working with these versions of the libraries. However, if you feel that performance may be better on other versions, feel free to upgrade/downgrade.

In [1]:
!pip install pandas==2.0.3
!pip install numpy==1.24.4
!pip install scikit-learn==1.3.2
!pip install rdkit
!pip install matplotlib
!pip install seaborn

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at 

In [34]:
import pandas as pd
print(f'pandas: {pd.__version__}')
import numpy as np
print(f'numpy: {np.__version__}')
import matplotlib.pyplot as plt
%matplotlib inline

pandas: 2.0.3
numpy: 1.24.4


### Loading the dataset
Next, we proceed to load the training data for the machine learning model. We performed in-depth analysis to choose the set of features on which the models perform the best (refer to the associated manuscript). Here, we will load the dataset with the set of features that gave us the best performance. If you are interested in training these models on a different set of features, you may directly work with the raw data, that can be found here: https://toxric.bioinforai.tech/home. Discussions pertaining to what these features are, and their physical meanings, are again redirected to the associated manuscript.

For loading the data we develop a function. The only possible issue that may arise here is with the 'path'. While the notebook has been provided with the training data in the same directory, if for some reason this is moved, you would need to specify the new path.  

Once the data has been loaded, we will also split it into training and testing data. We got the best results at a 80%/20% split, but the user is free to experiment with the split. The random_state ensures the same split everytime the model is trained. If you change the number here, there may be very small differences in the observed metrics.

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
def data(path):
    data_featurized = pd.read_csv(path) # modify accordingly
    y = data_featurized['Category']
    X = data_featurized.drop(['Category'], axis = 1)
    '''
    We got best results at a train-test-split of 80%/20%. 
    The user however, is free to experiment with the split. 

    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 14)

    return X_train, X_test, y_train, y_test

In [37]:
path = 'data_sampled_random_ADASYN_110_3class.csv'
X_train, X_test, y_train, y_test = data(path)

Okay, now we have the training data (X_train, y_train) and the testing data (X_test, y_test) on which our models will be trained and tested. For more context on how this data is prepared and processed, and the training process - we would again redirect you to the manuscript and its supporting information.

### Defining the machine learning model (and associated metrics)
Now it is time to define the machine learning model, and the metrics on which they will be evaluated. As a part of this study, several models were developed, trained and tested. Here, we write the code for the best performing model. We will not go into too much detail into why these models were chosen, how they were trained, and how the optimum hyperparameters were found - this has been covered to great detail in the manuscript.

We will not provide much context on the code here, we assume the reader has referred to the manuscript. Most of this code is available on GitHub, so you may find some additional code that is probably not being used in the present context

In [38]:
import pickle # will be used to save the model - so that it doesnt need to be trained everytime.
from sklearn.ensemble import RandomForestClassifier # the best performing model was the random forest classifier.
from sklearn.metrics import make_scorer, matthews_corrcoef,  balanced_accuracy_score, accuracy_score
# the metrics against which our model is evaluated.
from sklearn.model_selection import cross_validate 
from sklearn.model_selection import RepeatedStratifiedKFold

In [39]:
'''
Let us now define a function where the model training takes place. 

'''
def randomforestclassifier(X, y):
    '''
    Below, we define the scoring functions. For more context as to why these metrics are chosen, or how they are calculated, 
    please refer to the manuscript - where these concepts are discussed in some detail. 

    '''
    scoring = {'accuracy' : make_scorer(accuracy_score),
               'balanced_accuracy' : make_scorer(balanced_accuracy_score),
               'f1_macros' : 'f1_macro',
               'MCC' : make_scorer(matthews_corrcoef)}
    scores = []

    '''
    The hyperparameters inputted in the model in principle should yield the best metrics. Small tweaks may improve performance, but marginally at best.
    '''

    model = RandomForestClassifier(n_jobs = -1, random_state = 0, n_estimators = 725, 
                                   max_depth = 32, min_samples_split = 2, min_samples_leaf = 1)
    
    print('Starting cross-validation...')

    '''
    We perform a 5-fold cross-validation, with a single repeat - as is recommended for large datasets. 

    '''

    cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 1, random_state = 0)
    n_scores = cross_validate(model, X, y, scoring = scoring, cv = cv)
    print('Cross-validation completed...')
    print('Saving scores...')

    '''
    Once the cross-validation is complete, we can save the scores of the model. These scores will come in handy for some future analysis.
    '''

    scores.append(np.mean(n_scores['test_accuracy']))
    accuracy = n_scores['test_accuracy']
    scores.append(np.mean(n_scores['test_balanced_accuracy']))
    balanced_accuracy = n_scores['test_balanced_accuracy']
    scores.append(np.mean(n_scores['test_f1_macros']))
    f1 = n_scores['test_f1_macros']
    scores.append(np.mean(n_scores['test_MCC']))
    MCC = n_scores['test_MCC']
    print('Scores saved...')
    '''
    Finally, we fit the model on the training data. 
    '''
    model.fit(X, y)
    return scores, model, accuracy, balanced_accuracy, f1, MCC

In [40]:
'''
Let us bring it all together and run the model now. 
'''
def run_model(path):
    X_train, X_test, y_train, y_test = data(path)
    cross_validation_scores, model, accuracy, balanced_accuracy, f1, MCC = randomforestclassifier(X_train, y_train)
    '''
    Once the model is trained on the training data, it is necessary to see if it is able to retain its accuracy on the testing data.
    Moreover, we will extract useful metrics such as the true positive and false positive rates from these results - refer to the main manuscript for more context.
    '''
    y_pred = model.predict(X_test)
    test_score = accuracy_score(y_pred, y_test)
    print('Cross-validation scores are: ' + str(cross_validation_scores))
    print('Score on the test data is: ' + str(test_score))

    return cross_validation_scores, test_score, model, accuracy, balanced_accuracy, f1, MCC

In [41]:
cross_validation_scores, test_score, model, accuracy, balanced_accuracy, f1, MCC = run_model(path)

Starting cross-validation...
Cross-validation completed...
Saving scores...
Scores saved...
Cross-validation scores are: [0.8179217664367133, 0.8170271586227915, 0.8164748497334481, 0.7274077168577942]
Score on the test data is: 0.8332773109243697


Great! The model has been successfully trained. Our model has an accuracy of 81.8% on the training data, and 83.3% on the testing data. There may be marginal changes everytime the training process is carried out, assuming the hyperparameters are unchanged. If they are changed, there may be a drastic drop in performance.

We will now save the model so that we don't need to repeat the training everytime. The next time we need the code, we can simply load the saved model.

In [42]:
filename = 'final_model.sav'
pickle.dump(model, open(filename, 'wb'))

In principle, you need to train the model only once. After the model has been saved, you can directly load the model, and use it to make predictions on MOF structures. 

## Using the model to predict the toxicity of the MOF linker
Now that the model has been trained, we can easily load it to predict toxicity. Our model is accurate at capturing the toxicity of the MOF linker, while for the metallic centre, we would recommend following our discussion in the manuscript. Unfortunately in this notebook format, it would not be possible to run the code for the fragmentation of the MOF, as it requires openbabel, which cannot be setup with python bindings without more active involvement from the user. We thus operate under the assumption that you know the metal centre and the linker present in the MOF under investigation. If you want to use the fragmentation code, we heavily recommend using the format available on the GitHub repository.

Let us first load the model, after which we will use it on the MOF as specified by the user.

In [43]:
import pickle

In [44]:
def load_model(filepath):
    model = pickle.load(open(filepath, 'rb'))
    return model

In [45]:
filepath = 'final_model.sav'
model = load_model(filepath)

Now that the model has been loaded, let us featurise the linker and pass it to the model to predict the toxicity class. We demonstrate this here for the HKUST-1 MOF, however, you simply need to replace this with the SMILES representation of your MOF linker.

In [54]:
'''
Replace the canonical SMILES above with the SMILES representation of your MOF linker. 
There are several tools available online that can help you with this.
'''

canonical_SMILES = 'C1=C(C=C(C=C1C(=O)O)C(=O)O)C(=O)O'

Now that we know the linker - the next step would be to extract the 110 features that need to be provided as the input to the model.

In [55]:
from rdkit import Chem 
from rdkit.Chem import Descriptors  

In [56]:
def featurize(linker):
    try:
        m = Chem.MolFromSmiles(linker)
        EState_VSA1 = Descriptors.EState_VSA1(m) 
        EState_VSA2 = Descriptors.EState_VSA2(m)
        EState_VSA3 = Descriptors.EState_VSA3(m)
        EState_VSA4 = Descriptors.EState_VSA4(m)
        EState_VSA5 = Descriptors.EState_VSA5(m)
        EState_VSA6 = Descriptors.EState_VSA6(m)
        EState_VSA7 = Descriptors.EState_VSA7(m)
        EState_VSA8 = Descriptors.EState_VSA8(m)
        EState_VSA9 = Descriptors.EState_VSA9(m)
        EState_VSA10 = Descriptors.EState_VSA10(m)
        VSA_EState1 = Descriptors.VSA_EState1(m)
        VSA_EState2 = Descriptors.VSA_EState2(m)
        VSA_EState3 = Descriptors.VSA_EState3(m)
        VSA_EState5 = Descriptors.VSA_EState5(m)
        VSA_EState6 = Descriptors.VSA_EState6(m)
        VSA_EState7 = Descriptors.VSA_EState7(m)
        VSA_EState8 = Descriptors.VSA_EState8(m)
        VSA_EState9 = Descriptors.VSA_EState9(m)
        QED = Descriptors.qed(m)
        BertzCT = Descriptors.BertzCT(m)
        Chi0 = Descriptors.Chi0(m)
        Chi0n = Descriptors.Chi0n(m)
        Chi0v = Descriptors.Chi0v(m)
        Chi1 = Descriptors.Chi1(m)
        Chi1n = Descriptors.Chi1n(m)
        Chi1v = Descriptors.Chi1v(m)
        Chi2n = Descriptors.Chi2n(m)
        Chi3n = Descriptors.Chi3n(m)
        Chi3v = Descriptors.Chi3v(m)
        Chi4n = Descriptors.Chi4n(m)
        Chi4v = Descriptors.Chi4v(m)
        HallKierAlpha = Descriptors.HallKierAlpha(m)
        Kappa1 = Descriptors.Kappa1(m)
        Kappa2 = Descriptors.Kappa2(m)
        MinAbsPartialCharge = Descriptors.MinAbsPartialCharge(m)
        FpDensityMorgan2 = Descriptors.FpDensityMorgan2(m)
        FpDensityMorgan3 = Descriptors.FpDensityMorgan3(m)
        HeavyAtomMolWt = Descriptors.HeavyAtomMolWt(m)
        ExactMolWt = Descriptors.ExactMolWt(m)
        MolWt = Descriptors.MolWt(m)
        NumValenceElectrons = Descriptors.NumValenceElectrons(m)
        MaxPartialCharge = Descriptors.MaxPartialCharge(m)
        MolMR = Descriptors.MolMR(m)
        LabuteASA = Descriptors.LabuteASA(m)
        PEOE_VSA1 = Descriptors.PEOE_VSA1(m)
        PEOE_VSA2 = Descriptors.PEOE_VSA2(m)
        PEOE_VSA3 = Descriptors.PEOE_VSA3(m)
        PEOE_VSA9 = Descriptors.PEOE_VSA9(m)
        PEOE_VSA10 = Descriptors.PEOE_VSA10(m)
        PEOE_VSA12 = Descriptors.PEOE_VSA12(m)
        PEOE_VSA13 = Descriptors.PEOE_VSA13(m)
        PEOE_VSA14 = Descriptors.PEOE_VSA14(m)
        SMR_VSA1 = Descriptors.SMR_VSA1(m)
        SMR_VSA3 = Descriptors.SMR_VSA3(m)
        SMR_VSA4 = Descriptors.SMR_VSA4(m)
        SMR_VSA5 = Descriptors.SMR_VSA5(m)
        SMR_VSA6 = Descriptors.SMR_VSA6(m)
        SMR_VSA7 = Descriptors.SMR_VSA7(m)
        SMR_VSA9 = Descriptors.SMR_VSA9(m)
        SMR_VSA10 = Descriptors.SMR_VSA10(m)
        SlogP_VSA2 = Descriptors.SlogP_VSA2(m)
        SlogP_VSA3 = Descriptors.SlogP_VSA3(m)
        SlogP_VSA4 = Descriptors.SlogP_VSA4(m)
        SlogP_VSA5 = Descriptors.SlogP_VSA5(m)
        SlogP_VSA6 = Descriptors.SlogP_VSA6(m)
        SlogP_VSA8 = Descriptors.SlogP_VSA8(m)
        SlogP_VSA10 = Descriptors.SlogP_VSA10(m)
        TPSA = Descriptors.TPSA(m)
        MaxAbsEStateIndex = Descriptors.MaxAbsEStateIndex(m)
        MaxEStateIndex = Descriptors.MaxEStateIndex(m)
        MinEStateIndex = Descriptors.MinEStateIndex(m)
        MinAbsEStateIndex = Descriptors.MinAbsEStateIndex(m)
        fr_Al_OH = Descriptors.fr_Al_OH(m)
        fr_Al_OH_noTert = Descriptors.fr_Al_OH_noTert(m)
        fr_ArN = Descriptors.fr_ArN(m)
        fr_Ar_N = Descriptors.fr_Ar_N(m)
        fr_C_O = Descriptors.fr_C_O(m)
        fr_C_O_noCOO = Descriptors.fr_C_O_noCOO(m)
        fr_NH0 = Descriptors.fr_NH0(m)
        fr_Ndealkylation1 = Descriptors.fr_Ndealkylation1(m)
        fr_Ndealkylation2 = Descriptors.fr_Ndealkylation2(m)
        fr_allylic_oxid = Descriptors.fr_allylic_oxid(m)
        fr_amide = Descriptors.fr_amide(m)
        fr_aniline = Descriptors.fr_aniline(m)
        fr_benzene = Descriptors.fr_benzene(m)
        fr_bicyclic = Descriptors.fr_bicyclic(m)
        fr_epoxide = Descriptors.fr_epoxide(m)
        fr_ester = Descriptors.fr_ester(m)
        fr_ether = Descriptors.fr_ether(m)
        fr_imide = Descriptors.fr_imide(m)
        fr_lactam = Descriptors.fr_lactam(m)
        fr_para_hydroxylation = Descriptors.fr_para_hydroxylation(m)
        fr_piperdine = Descriptors.fr_piperdine(m)
        fr_quatN = Descriptors.fr_quatN(m)
        FractionCSP3 = Descriptors.FractionCSP3(m)
        HeavyAtomCount = Descriptors.HeavyAtomCount(m)
        NHOHCount = Descriptors.NHOHCount(m)
        NOCount = Descriptors.NOCount(m)
        NumAliphaticCarbocycles = Descriptors.NumAliphaticCarbocycles(m)
        NumAliphaticHeterocycles = Descriptors.NumAliphaticHeterocycles(m)
        NumAliphaticRings = Descriptors.NumAliphaticRings(m)
        NumAromaticCarbocycles = Descriptors.NumAromaticCarbocycles(m)
        NumAromaticHeterocycles = Descriptors.NumAromaticHeterocycles(m)
        NumAromaticRings = Descriptors.NumAromaticRings(m) 
        NumHDonors = Descriptors.NumHDonors(m)
        NumHeteroatoms = Descriptors.NumHeteroatoms(m)
        NumRotatableBonds = Descriptors.NumRotatableBonds(m)
        NumSaturatedCarbocycles = Descriptors.NumSaturatedCarbocycles(m)	
        NumSaturatedHeterocycles = Descriptors.NumSaturatedHeterocycles(m)	
        NumSaturatedRings = Descriptors.NumSaturatedRings(m)

        features = {'EState_VSA1' : EState_VSA1,
                    'EState_VSA2' : EState_VSA2,
                    'EState_VSA3' : EState_VSA3,
                    'EState_VSA4' : EState_VSA4,
                    'EState_VSA5' : EState_VSA5,
                    'EState_VSA6' : EState_VSA6,
                    'EState_VSA7' : EState_VSA7,
                    'EState_VSA8' : EState_VSA8,
                    'EState_VSA9' : EState_VSA9,
                    'EState_VSA10' : EState_VSA10,
                    'VSA_EState1' : VSA_EState1,
                    'VSA_EState2' : VSA_EState2,
                    'VSA_EState3' : VSA_EState3,
                    'VSA_EState5' : VSA_EState5,
                    'VSA_EState6' : VSA_EState6,
                    'VSA_EState7' : VSA_EState7,
                    'VSA_EState8' : VSA_EState8,
                    'VSA_EState9' : VSA_EState9,
                    'QED' : QED,
                    'BertzCT' : BertzCT,
                    'Chi0' : Chi0,
                    'Chi0n' : Chi0n,
                    'Chi0v' : Chi0v,
                    'Chi1' : Chi1,
                    'Chi1n' : Chi1n,
                    'Chi1v' : Chi1v,
                    'Chi2n' : Chi2n,
                    'Chi3n' : Chi3n,
                    'Chi3v' : Chi3v,
                    'Chi4n' : Chi4n,
                    'Chi4v' : Chi4v,
                    'HallKierAlpha' : HallKierAlpha,
                    'Kappa1' : Kappa1,
                    'Kappa2' : Kappa2,
                    'MinAbsPartialCharge' : MinAbsPartialCharge,
                    'FpDensityMorgan2' : FpDensityMorgan2,
                    'FpDensityMorgan3' : FpDensityMorgan3,
                    'HeavyAtomMolWt' : HeavyAtomMolWt,
                    'ExactMolWt': ExactMolWt,
                    'MolWt' : MolWt,
                    'NumValenceElectrons' : NumValenceElectrons,
                    'MaxPartialCharge' : MaxPartialCharge,
                    'MolMR' : MolMR,
                    'LabuteASA' : LabuteASA,
                    'PEOE_VSA1' : PEOE_VSA1,
                    'PEOE_VSA2' : PEOE_VSA2,
                    'PEOE_VSA3' : PEOE_VSA3,
                    'PEOE_VSA9' : PEOE_VSA9,
                    'PEOE_VSA10' : PEOE_VSA10,
                    'PEOE_VSA12' : PEOE_VSA12,
                    'PEOE_VSA13' : PEOE_VSA13,
                    'PEOE_VSA14' : PEOE_VSA14,
                    'SMR_VSA1' : SMR_VSA1,
                    'SMR_VSA3' : SMR_VSA3,
                    'SMR_VSA4' : SMR_VSA4,
                    'SMR_VSA5' : SMR_VSA5,
                    'SMR_VSA6' : SMR_VSA6,
                    'SMR_VSA7' : SMR_VSA7,
                    'SMR_VSA9' : SMR_VSA9,
                    'SMR_VSA10' : SMR_VSA10,
                    'SlogP_VSA2' : SlogP_VSA2,
                    'SlogP_VSA3' : SlogP_VSA3,
                    'SlogP_VSA4' : SlogP_VSA4,
                    'SlogP_VSA5' : SlogP_VSA5,
                    'SlogP_VSA6' : SlogP_VSA6,
                    'SlogP_VSA8' : SlogP_VSA8,
                    'SlogP_VSA10' : SlogP_VSA10,
                    'TPSA' : TPSA,
                    'MaxAbsEStateIndex' : MaxAbsEStateIndex,
                    'MaxEStateIndex' : MaxEStateIndex,
                    'MinEStateIndex' : MinEStateIndex,
                    'MinAbsEStateIndex' : MinAbsEStateIndex,
                    'fr_Al_OH' : fr_Al_OH,
                    'fr_Al_OH_noTert' : fr_Al_OH_noTert,
                    'fr_ArN' : fr_ArN,
                    'fr_Ar_N' : fr_Ar_N,
                    'fr_C_O' : fr_C_O,
                    'fr_C_O_noCOO' : fr_C_O_noCOO,
                    'fr_NH0' : fr_NH0,
                    'fr_Ndealkylation1' : fr_Ndealkylation1,
                    'fr_Ndealkylation2' : fr_Ndealkylation2,
                    'fr_allylic_oxid' : fr_allylic_oxid,
                    'fr_amide' : fr_amide,
                    'fr_aniline' : fr_aniline,
                    'fr_benzene' : fr_benzene,
                    'fr_bicyclic' : fr_bicyclic,
                    'fr_epoxide' : fr_epoxide,
                    'fr_ester' : fr_ester,
                    'fr_ether' : fr_ether,
                    'fr_imide' : fr_imide,
                    'fr_lactam' : fr_lactam,
                    'fr_para_hydroxylation' : fr_para_hydroxylation,
                    'fr_piperdine': fr_piperdine,
                    'fr_quatN' : fr_quatN,
                    'FractionCSP3' : FractionCSP3,
                    'HeavyAtomCount' : HeavyAtomCount,
                    'NHOHCount' : NHOHCount,
                    'NOCount' : NOCount,
                    'NumAliphaticCarbocycles' : NumAliphaticCarbocycles,
                    'NumAliphaticHeterocycles' : NumAliphaticHeterocycles,
                    'NumAliphaticRings' : NumAliphaticRings,
                    'NumAromaticCarbocycles' : NumAromaticCarbocycles,
                    'NumAromaticHeterocycles' : NumAromaticHeterocycles,
                    'NumAromaticRings' : NumAromaticRings,
                    'NumHDonors' : NumHDonors,
                    'NumHeteroatoms' : NumHeteroatoms,
                    'NumRotatableBonds' : NumRotatableBonds,
                    'NumSaturatedCarbocycles' : NumSaturatedCarbocycles,
                    'NumSaturatedHeterocycles' : NumSaturatedHeterocycles,
                    'NumSaturatedRings' : NumSaturatedRings}
    except:
        print('Error with featurization')
    return features

The featurization procedure is fairly robust. In rare cases, it fails. The most likely reason for it to fail is that there are some errors in the SMILES representation of the linker - so it would be best to check this.

In [57]:
features = featurize(canonical_SMILES)
X = pd.DataFrame([features], columns=features.keys())

Cool, now let us make a prediction of the toxicity class.

In [58]:
prediction = model.predict(X)

if prediction == -1:
    print('Toxicity Class: Fatal')
elif prediction == 0:
    print('Toxicity Class: Toxic')
else:
    print('Toxicity Class: Safe')

Toxicity Class: Safe


### Regarding the metallic centre of the MOF

For the metallic centre of the MOF, there are no algorithms per se - instead, we provide an in-depth discussion on their toxicity, including principles to guide the selection of optimum metallic centres in the associated manuscript. The reader is advised to refer to the source.